# 007 Splitter Pattern

In [1]:
from enum import StrEnum
from typing import List, Optional, Union
from jsonata import Jsonata
from pydantic import BaseModel, Field
from typing import List, Dict, Any

In [2]:
# Import required libraries and modules
import uuid

from rustic_ai.core.agents.eip.aggregating_agent import (
    AggregatedMessages,
    AggregatingAgent,
    AggregatorConf,
    CountingAggregator,
    DictCollector,
)
from rustic_ai.core.agents.eip.basic_wiring_agent import BasicWiringAgent
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name
from rustic_ai.core.utils.jexpr import JExpr, JObj, JxScript

In [3]:
class PurchaseOrderRequest(BaseModel):
    order_id: str
    items: List[Dict[str, Any]]
    customer: str

In [4]:
class ItemProcessingResult(BaseModel):
    order_id: str
    item_id: str
    status: str
    processed_price: float

In [5]:
from rustic_ai.core.agents.eip.splitter_agent import SplitterConf, SplitterAgent, JsonataSplitter, FormatSelector, FormatSelectorStrategies

In [6]:
from rustic_ai.core.utils.jx import JExpr
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name

In [7]:
splitter_conf = SplitterConf(
    splitter=JsonataSplitter(
        expression= JExpr('$map(payload.items, function($v) {$merge([$v, {"order_id": payload.order_id}, {"customer": payload.customer}] ) } )').serialize()
    ),
    format_selector=FormatSelector(
        strategy=FormatSelectorStrategies.FIXED,
        fixed_format=get_qualified_class_name(ItemProcessingResult)
    ),
    topics=["purchase_orders"]
)

In [8]:
splitter_agent = (
    AgentBuilder(SplitterAgent)
    .set_id("SplitterAgent")
    .set_name("Purchase Order Splitter")
    .set_description("Splits a PurchaseOrderRequest into multiple ItemRequest messages")
    .set_properties(splitter_conf)
    .add_additional_topic("purchase_orders")
    .listen_to_default_topic(False)
    .build_spec()
)

In [9]:
from rustic_ai.core.utils.json_utils import JsonDict

In [10]:
split_route = (
    RouteBuilder(splitter_agent)
    .on_message_format(PurchaseOrderRequest)
    .set_functional_transformer(
        functional_xform=JxScript(
            JObj({
                "format": get_qualified_class_name(JsonDict),
                "payload": JExpr("payload")
            })
        )
    )
    .set_destination_topics("purchase_orders")
    .build()
)


In [11]:
guild_builder = (
    GuildBuilder(
        guild_id="SplitterGuild",
        guild_name="Purchase Order Split Demo",
        guild_description="Demonstrates splitting of messages using SplitterAgent"
    )
    .add_agent_spec(splitter_agent)
    .add_route(split_route)
)

In [12]:
import os
from rustic_ai.core.guild.metastore.database import Metastore

In [13]:
# === Bootstrap ===

db = "sqlite:///splitter_demo.db"
if os.path.exists("splitter_demo.db"):
    os.remove("splitter_demo.db")   

Metastore.initialize_engine(db)
Metastore.get_engine(db)
Metastore.create_db()

guild = guild_builder.bootstrap(metastore_database_url=db, organization_id="demo-org")


In [14]:
from rustic_ai.core.utils import GemstoneGenerator, Priority

In [15]:
generator = GemstoneGenerator(17)

In [16]:
# === Add Probe Agent ===
import time

time.sleep(2)

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("TestProbe")
    .set_name("Test Probe Agent")
    .set_description("Monitors the splitter flow")
    .add_additional_topic("purchase_orders")
    .add_additional_topic("split_order_items")
    .add_additional_topic("item_processing_results")
    .build()
)

guild._add_local_agent(probe_agent)

In [17]:
test_order = PurchaseOrderRequest(
    order_id="PO-12345",
    customer="ACME Corp",
    items=[
        {"id": "item-001", "quantity": 2},
        {"id": "item-002", "quantity": 1}
    ]
)


In [18]:
from rustic_ai.core.messaging.core.message import AgentTag, Message

In [19]:
probe_agent.publish_dict(
    topic="purchase_orders",
    payload=test_order.model_dump(),
    format=get_qualified_class_name(PurchaseOrderRequest),
    msg_id=generator.get_id(Priority.NORMAL)
)

In [25]:
probe_agent.get_messages()

[]

In [21]:
from rustic_ai.core.guild.dsl import GuildTopics

In [24]:
probe_agent._client._messaging.get_messages_for_topic_since(
    topic="purchase_orders",
    msg_id_since=0
)

[Message(sender=AgentTag(id='TestProbe', name='Test Probe Agent'), topics='purchase_orders', recipient_list=[], payload={'order_id': 'PO-12345', 'items': [{'id': 'item-001', 'quantity': 2}, {'id': 'item-002', 'quantity': 1}], 'customer': 'ACME Corp'}, format='__main__.PurchaseOrderRequest', in_response_to=None, thread=[9566072168073203712], conversation_id=None, forward_header=None, routing_slip=None, message_history=[], ttl=None, is_error_message=False, traceparent=None, session_state=None, topic_published_to='purchase_orders', enrich_with_history=0, id=9566072168073203712, priority=<Priority.NORMAL: 4>, timestamp=1754237278343, current_thread_id=9566072168073203712, root_thread_id=9566072168073203712)]

In [31]:
spec = guild_builder.build_spec()
dict_obj = spec.model_dump()

In [32]:
import yaml

In [33]:
yaml_string = yaml.dump(dict_obj)

In [36]:
with open('/home/nihal/Projects/ai-platform/rustic-ai/examples/notebooks/eip/007_splitter.yaml', 'w') as f:
    yaml.dump(dict_obj, f)

In [34]:
yaml_string

'agents:\n- act_only_when_tagged: false\n  additional_topics:\n  - purchase_orders\n  class_name: rustic_ai.core.agents.eip.splitter_agent.SplitterAgent\n  dependency_map: {}\n  description: Splits a PurchaseOrderRequest into multiple ItemRequest messages\n  id: SplitterAgent\n  listen_to_default_topic: false\n  name: Purchase Order Splitter\n  predicates: {}\n  properties:\n    delimiter: \',\'\n    format_selector:\n      fixed_format: __main__.ItemProcessingResult\n      format_list: null\n      jsonata_expr: null\n      strategy: !!python/object/apply:rustic_ai.core.agents.eip.splitter_agent.FormatSelectorStrategies\n      - fixed\n    splitter:\n      expression: \'$map(payload.items, function($v) {$merge([$v, {"order_id": payload.order_id},\n        {"customer": payload.customer}] ) } )\'\n      split_type: jsonata\n    topics:\n    - purchase_orders\n  qos:\n    latency: null\n    retry_count: null\n    timeout: null\n  resources:\n    custom_resources: {}\n    num_cpus: null\n 

In [28]:
# from rustic_ai.core.agents.system.models import StopGuildRequest


# probe_agent.publish_with_guild_route(
#     topic=GuildTopics.SYSTEM_TOPIC, payload=StopGuildRequest(guild_id=guild.id)
# )  # Send the test request to start the flow

# guild.shutdown()